In [3]:
%load_ext autoreload
%autoreload 1

#  https://colab.research.google.com/notebooks/data_table.ipynb
from google.colab import data_table
data_table.enable_dataframe_formatter()
from google.colab import drive
drive.mount('/content/gdrive')

# https://stackoverflow.com/questions/52733786/how-to-import-custom-modules-in-google-colab
import sys
sys.path.append('/content/gdrive/MyDrive/SEMRUSH/')

%aimport my

import sys
import numpy as np
import pandas as pd
import os
import gc
import seaborn as sns
import matplotlib.pyplot as plt
import importlib
import pickle

import my
from my import p

pd.set_option("max_colwidth", 35)
pd.set_option("display.precision", 1)
pd.options.display.float_format = "{:.3f}".format
# pd.set_option("display.max_rows", 5)
# pd.reset_option("display.max_rows")

from IPython.display import HTML, display

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
     
 # номер или название эксперимента
# n_exp = '1/'
dir_out = "/content/gdrive/MyDrive/SEMRUSH/out/"
dir_data = '/content/gdrive/MyDrive/SEMRUSH/data/'
os.makedirs(dir_out, exist_ok = True)
RANDOM_STATE = 34

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
test = pd.read_parquet(dir_data+'test_data.p')
p(test.shape)
test[:3]

(6077489, 26) 
 ~


,event_group_id,device_id,timestamp,domain,url,pos,ff_count_slash,ff_count_subdomain,root_domain,ltd_domain,...,ff_count_slash_target,ff_count_subdomain_target,cat_domain,cat_target,pair_target,pair_root_target,pair_ltd_target,pair_cat_target,pair_url_target,pair_path_url_target
0,1,327782,1168864757,194254,4242956,0,5,2,161321,231,...,2,2,65,65,11805900194254,9837450161321,2310000231,650000065,40397604242956,28808603253601
1,1,327782,1168864762,1093395,3141193,1,1,3,910835,231,...,2,2,65,65,11805901093395,9837450910835,2310000231,650000065,40397603141193,28808600547807
2,1,327782,1168864763,717284,1881192,2,1,2,597474,231,...,2,2,65,65,11805900717284,9837450597474,2310000231,650000065,40397601881192,28808601806665


In [3]:
FOLD = 2
sts = my.load_pickle(dir_data+f'sts_split_3_fold_{FOLD}.pickle')

In [4]:
test = my.add_stats_features(test, sts)
del sts
p(test.isna().sum().sum())
test[:3]

0 
 ~


,event_group_id,device_id,timestamp,domain,url,pos,ff_count_slash,ff_count_subdomain,root_domain,ltd_domain,...,ff_count_ltd_ref_ltd_domain,ff_count_cat_ref_cat_domain,ff_count_url_ref_url,ff_count_path_url_ref_path_url,ff_prob_ref,ff_prob_ref_root,ff_prob_ref_ltd,ff_prob_ref_cat,ff_prob_ref_url,ff_prob_ref_path_url
0,1,327782,1168864757,194254,4242956,0,5,2,161321,231,...,260.000,110.000,1.000,1.000,0.000,0.000,0.932,0.940,0.000,0.000
1,1,327782,1168864762,1093395,3141193,1,1,3,910835,231,...,260.000,110.000,80.000,24.000,0.000,0.000,0.932,0.940,0.000,0.000
2,1,327782,1168864763,717284,1881192,2,1,2,597474,231,...,260.000,110.000,217.000,1759.000,0.000,0.000,0.932,0.940,0.000,0.000


In [5]:
del_cols = ['root_domain', 'ltd_domain', 'root_target', 'ltd_target', 'path_url',
       'path_url_target', 'cat_domain', 'cat_target',
       'pair_root_target', 'pair_ltd_target', 'pair_cat_target',
       'pair_url_target', 'pair_path_url_target']
test.drop(columns=del_cols, inplace=True)
test = my.time_features(test)
test = test.sort_values(['event_group_id','pos']).reset_index(drop=True)

In [6]:
test = my.merge_stat_pair_counts(test,col='domain',\
                                merge_col='pair_target')
test = test.loc[test.pos != 10].reset_index(drop=True)
gc.collect()
test.shape, test.isna().sum().sum()

pair_domain_tar


((5524990, 55), 0)

In [7]:
%%capture
!pip install catboost

In [8]:
from catboost import CatBoostClassifier

fs = my.select_cols(test,['ff'])
X_test = test[fs]

test_df = test[['event_group_id']].copy()
test_df['probs'] = 0

for i in range(3):
    print('cb = ',i)
    
    cb = CatBoostClassifier()
    cb.load_model('/content/gdrive/MyDrive/SEMRUSH/models/'\
                  +f'cb_fold_{FOLD}_{i}.cbm')


    test_df['probs'] += cb.predict_proba(X_test)[:,1]
    
test_df.to_parquet(dir_data+f'test_df{FOLD}.p')

cb =  0
cb =  1
cb =  2


In [4]:
df0 = pd.read_parquet(dir_data+f'test_df0.p')
df1 = pd.read_parquet(dir_data+f'test_df1.p')
df2 = pd.read_parquet(dir_data+f'test_df2.p')
df0

,event_group_id,probs
0,1,0.021
1,1,0.001
2,1,0.001
3,1,0.783
4,1,0.106
...,...,...
5524985,1738010,0.000
5524986,1738010,0.000
5524987,1738010,0.000
5524988,1738010,0.463


In [5]:
df0['probs'] = df0['probs'] + df1['probs']+ df2['probs']
df0

,event_group_id,probs
0,1,0.060
1,1,0.002
2,1,0.003
3,1,4.223
4,1,0.290
...,...,...
5524985,1738010,0.000
5524986,1738010,0.001
5524987,1738010,0.001
5524988,1738010,1.216


In [7]:
def to_sub(test_df,name):
    test_df = test_df.groupby('event_group_id')['probs'].agg(
        lambda x: x.argmax())
    test_df.name = 'referrer_num'
    test_df.to_csv(f'{name}')
    print(f'Сохранено в файл: {name}')

to_sub(df0,'sub100.csv')

Сохранено в файл: sub100.csv
